In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [3]:
from torchvision import transforms

def get_mean_std(loader):
    # Compute the mean and standard deviation of all pixels in the dataset
    num_pixels = 0
    mean = 0.0
    std = 0.0
    for images, _ in loader:
        batch_size, num_channels, height, width = images.shape
        num_pixels += batch_size * height * width
        mean += images.mean(axis=(0, 2, 3)).sum()
        std += images.std(axis=(0, 2, 3)).sum()

    mean /= num_pixels
    std /= num_pixels

    return mean, std

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

batch_size = 64
train_loader = torch.utils.data.DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)
mean, std = get_mean_std(train_loader)


In [4]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

training = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=data_transforms,
)

testing = datasets.MNIST(
    root="data",
    train=True,
    download=False,
    transform=data_transforms,
)


In [5]:
images, labels = next(iter(train_loader))  # Get one batch
print(images.shape, labels.shape)  # Expected: (batch_size, 1, 28, 28), (batch_size)

torch.Size([64, 1, 28, 28]) torch.Size([64])


In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
import os
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
from torch import nn

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [11]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------------------------')
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print('Done!')

Epoch 1
--------------------------------------
loss: 2.298427  [   64/60000]
loss: 2.302478  [ 6464/60000]
loss: 2.296855  [12864/60000]
loss: 2.284636  [19264/60000]
loss: 2.287372  [25664/60000]
loss: 2.289424  [32064/60000]
loss: 2.279428  [38464/60000]
loss: 2.275830  [44864/60000]
loss: 2.258471  [51264/60000]
loss: 2.259295  [57664/60000]
Test Error: 
 Accuracy: 31.6%, Avg loss: 2.255784 

Epoch 2
--------------------------------------
loss: 2.259660  [   64/60000]
loss: 2.256246  [ 6464/60000]
loss: 2.243673  [12864/60000]
loss: 2.243333  [19264/60000]
loss: 2.216917  [25664/60000]
loss: 2.219338  [32064/60000]
loss: 2.210723  [38464/60000]
loss: 2.204393  [44864/60000]
loss: 2.195997  [51264/60000]
loss: 2.213048  [57664/60000]
Test Error: 
 Accuracy: 49.0%, Avg loss: 2.189260 

Epoch 3
--------------------------------------
loss: 2.188877  [   64/60000]
loss: 2.151811  [ 6464/60000]
loss: 2.182991  [12864/60000]
loss: 2.150708  [19264/60000]
loss: 2.138758  [25664/60000]
loss:

In [13]:
torch.save(model.state_dict(), 'mnist-model.pth')
print('Model saved successfully!')

Model saved successfully!
